In [5]:
import pandas as pd

file_url = 'https://huggingface.co/datasets/Sachinkelenjaguri/Resume_dataset/resolve/main/UpdatedResumeDataSet.csv'

df = pd.read_csv(file_url)
print(df)

         Category                                             Resume
0    Data Science  Skills * Programming Languages: Python (pandas...
1    Data Science  Education Details \nMay 2013 to May 2017 B.E  ...
2    Data Science  Areas of Interest Deep Learning, Control Syste...
3    Data Science  Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4    Data Science  Education Details \n MCA   YMCAUST,  Faridabad...
..            ...                                                ...
957       Testing  Computer Skills: â¢ Proficient in MS office (...
958       Testing  â Willingness to accept the challenges. â ...
959       Testing  PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne...
960       Testing  COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...
961       Testing  Skill Set OS Windows XP/7/8/8.1/10 Database MY...

[962 rows x 2 columns]
Resume Dataset shape:  (962, 2)


In [41]:
DOMAINS = ['Java Developer', 'Python Developer']

resume_dataset = []

for index, row in df.iterrows():
    domain = row["Category"]
    if (domain in DOMAINS):
      resume_dataset.append([domain, row["Resume"]])

print(resume_dataset)

[['Java Developer', 'Education Details \nAugust 2010 to May 2017 BE Electronics & Communication Jabalpur, Madhya Pradesh Takshshila institute of technology\nJava developer \n\n\nSkill Details \nJava, Javascript,- Exprience - 6 monthsCompany Details \ncompany - Wab It Softwere Pvt.  Ltd.\ndescription - Jr. Java Developer'], ['Java Developer', 'Technical Skills (Trained / Project Acquired Skills) â\x80¢ Languages Known: C, C++, J2EE, Spring, Hibernate. â\x80¢ Testing: Functional Testing, Manual Testing. â\x80¢ Operating Systems: Windows & Linux. â\x80¢ Packages: MS-office. â\x80¢ Databases: My SQL & PostgreSQL.Education Details \nJanuary 2013 to January 2017 B.E Computer Science and Engineering  RTMNU University - Jhulelal Institute of Technology\nJanuary 2011 to January 2013 HSC  Nagpur, Maharashtra Maharashtra State Board - Dayanand Arya Kanya Junior College\nJanuary 2010 to January 2011 SSC  Nagpur, Maharashtra Maharashtra State Board\nJava Developer \n\nJava Developer - Inouvelle Ven

In [42]:
print("Number of filtered out resume: ", len(resume_dataset))

Number of filtered out resume:  132


In [29]:
## Importing necessary libraries
!pip install langchain-google-vertexai
!pip install -U langgraph
!pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python nomic[local]

In [51]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_google_vertexai import ChatVertexAI
from langgraph.graph import StateGraph, END

from typing import TypedDict

class GraphInput(TypedDict):
    question: str


class GraphOutput(TypedDict):
    resume: str


class ResumeGraphState(TypedDict):
    question: str
    resume: str


base_prompt = """You are a Resume creator which is capable of creating sample resumes for a given domain.
The domains will be technical like Java Developer, Data Science, DevOps Engineer etc.
You should be able to tailor a strong resume for the provided domain.

Make it relevent to the domain.

Pay attention to the examples below. These are good examples. Generate future resumes in the style of the examples below."""

def create_resume(state: ResumeGraphState):
  messages = [SystemMessage(content = base_prompt)]
  java_dev_examples = 0
  python_dev_examples = 0

  for resume in resume_dataset:
    if java_dev_examples == 5 and python_dev_examples == 5:
      break

    if resume[0] == 'Java Developer' and java_dev_examples < 5:
      human_question = "Create a resume for domain: Java Developer"
      messages.append(HumanMessage(content = human_question))
      messages.append(AIMessage(content = resume[1]))
      java_dev_examples+=1

    if resume[0] == 'Python Developer' and python_dev_examples < 5:
      human_question = "Create a resume for domain: Python Developer"
      messages.append(HumanMessage(content = human_question))
      messages.append(AIMessage(content = resume[1]))
      python_dev_examples+=1

  messages.append(HumanMessage(content = state["question"]))
  llm = ChatVertexAI(model_name="gemini-pro")
  resume = llm.invoke(messages)

  return {"resume": resume.content}


resume_workflow = StateGraph(ResumeGraphState, input=GraphInput, output=GraphOutput)
resume_workflow.add_node(create_resume)

resume_workflow.set_entry_point("create_resume")
resume_workflow.add_edge("create_resume", END)

resume_graph = resume_workflow.compile()

In [52]:
created_resume = resume_graph.invoke(
    {"question": 'Create a resume for domain: Software Engineer'},
    config={"configurable": {"thread_id": 42}}
)

print(created_resume)

{'resume': "Education Details \nJune 2013 MCA - Anna University Chennai, Tamil Nadu - University Department of Computer Science\nJune 2009 BE ECE - Madras University Chennai, Tamil Nadu - St.Peter's Engineering College\nApril 2001    HSC - State Board Chennai, Tamil Nadu - Holy Angels School\nMarch 1999    SSLC - State Board Chennai, Tamil Nadu - Bhavans Rajaji Vidyashram\nSoftware Engineer - Quality Assurance - Test engineer - Automation\n\nSoftware engineer - Infosys\nSkill Details \nHTML - Exprience - 2 years - Working Knowledge\nCSS - Exprience - 2 years - Working Knowledge\nSQL - Exprience - 2 years - Working Knowledge\nGIT - Exprience - 2 years - Working Knowledge\nJAVA - Exprience - 2 years - Intermediate\nPYTHON - Exprience - 2 years - Intermediate\nREST- Exprience - 2 years - Intermediate\nSelenium Webdriver - Exprience - 2 years - Intermediate\nJenkins - Exprience - 2 years - Intermediate\nCucumber BDD - Exprience - 2 years - Intermediate\nJIRA - Exprience - 2 years - Interme